In [ ]:
"""
COCO Dataset Splitting with Supervision 📊
===========================================

This notebook demonstrates how to efficiently split COCO format datasets
for machine learning model training, validation, and testing.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/roboflow/supervision/blob/develop/examples/notebooks/coco_dataset_splitting.ipynb)
"""

In [ ]:
# Cell 1: Installation and Imports
print("🔧 Installing required packages...")

# Install supervision and dependencies
!pip install supervision pycocotools pillow

# Import required libraries
import os
import json
import numpy as np
from pathlib import Path
import supervision as sv
from PIL import Image
import matplotlib.pyplot as plt

print("✅ Setup complete!")


In [ ]:
# Cell 2: Download Sample Dataset
print("📥 Setting up sample COCO dataset...")

# Create sample COCO dataset for demonstration
sample_data = {
    "info": {
        "description": "Sample COCO Dataset for Splitting Demo",
        "version": "1.0",
        "year": 2024,
        "contributor": "Supervision Community",
        "date_created": "2024-01-01T00:00:00+00:00"
    },
    "licenses": [
        {
            "id": 1,
            "name": "Sample License",
            "url": "https://github.com/roboflow/supervision"
        }
    ],
    "categories": [
        {"id": 1, "name": "person", "supercategory": "person"},
        {"id": 2, "name": "bicycle", "supercategory": "vehicle"},
        {"id": 3, "name": "car", "supercategory": "vehicle"},
        {"id": 4, "name": "dog", "supercategory": "animal"}
    ],
    "images": [],
    "annotations": []
}

# Generate sample images and annotations
np.random.seed(42)  # For reproducible results
num_images = 100

print(f"🖼️ Generating {num_images} sample images with annotations...")

# Create images directory
images_dir = Path("sample_dataset/images")
images_dir.mkdir(parents=True, exist_ok=True)

annotation_id = 1

for img_id in range(1, num_images + 1):
    # Create sample image metadata
    img_info = {
        "id": img_id,
        "file_name": f"sample_{img_id:03d}.jpg",
        "width": 640,
        "height": 480,
        "date_captured": "2024-01-01T12:00:00+00:00"
    }
    sample_data["images"].append(img_info)
    
    # Create a simple colored image for demonstration
    img = Image.new('RGB', (640, 480), 
                    color=(np.random.randint(100, 255), 
                           np.random.randint(100, 255), 
                           np.random.randint(100, 255)))
    img.save(images_dir / img_info["file_name"])
    
    # Generate random annotations for this image
    num_annotations = np.random.randint(1, 5)  # 1-4 annotations per image
    
    for _ in range(num_annotations):
        # Random bounding box
        x = np.random.randint(0, 500)
        y = np.random.randint(0, 350)
        w = np.random.randint(50, 140)
        h = np.random.randint(50, 130)
        
        annotation = {
            "id": annotation_id,
            "image_id": img_id,
            "category_id": np.random.randint(1, 5),  # Random category
            "bbox": [x, y, w, h],
            "area": w * h,
            "iscrowd": 0,
            "segmentation": []  # Empty for bounding box only
        }
        sample_data["annotations"].append(annotation)
        annotation_id += 1

# Save annotations file
annotations_file = Path("sample_dataset/annotations.json")
with open(annotations_file, 'w') as f:
    json.dump(sample_data, f, indent=2)

print(f"✅ Created sample dataset:")
print(f"   📁 {len(sample_data['images'])} images")
print(f"   🏷️ {len(sample_data['annotations'])} annotations") 
print(f"   📂 {len(sample_data['categories'])} categories")


In [ ]:
# Cell 3: Basic Dataset Splitting
print("✂️ Splitting COCO dataset with Supervision...")

# Perform basic dataset split
stats = sv.split_coco_dataset(
    annotations_path="sample_dataset/annotations.json",
    images_directory="sample_dataset/images",
    output_directory="dataset_splits",
    val_percentage=0.2,    # 20% for validation
    test_percentage=0.1,   # 10% for testing (70% for training)
    random_state=42,       # For reproducible splits
    verify_images=True     # Verify image files exist
)

print("📊 Split Statistics:")
for split_name, split_stats in stats.items():
    if split_name != 'total':
        print(f"   {split_name.capitalize()}: {split_stats['images']} images, {split_stats['annotations']} annotations")

print(f"   Total: {stats['total']['images']} images, {stats['total']['annotations']} annotations")


In [ ]:
# Cell 4: Visualize Split Distribution
print("📈 Visualizing dataset splits...")

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Images distribution
splits = ['Train', 'Val', 'Test']
image_counts = [stats['train']['images'], stats['val']['images'], stats['test']['images']]
colors = ['#2E8B57', '#FF6347', '#4169E1']

ax1.pie(image_counts, labels=splits, colors=colors, autopct='%1.1f%%', startangle=90)
ax1.set_title('Image Distribution Across Splits')

# Annotations distribution
annotation_counts = [stats['train']['annotations'], stats['val']['annotations'], stats['test']['annotations']]

ax2.bar(splits, annotation_counts, color=colors)
ax2.set_title('Annotation Count by Split')
ax2.set_ylabel('Number of Annotations')
ax2.tick_params(axis='x', rotation=0)

# Add value labels on bars
for i, count in enumerate(annotation_counts):
    ax2.text(i, count + max(annotation_counts) * 0.01, str(count), 
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
# Cell 5: Advanced Splitting Options
print("⚙️ Demonstrating advanced splitting options...")

# Split with custom parameters
advanced_stats = sv.split_coco_dataset(
    annotations_path="sample_dataset/annotations.json",
    images_directory="sample_dataset/images",
    output_directory="advanced_splits",
    val_percentage=0.15,         # 15% validation
    test_percentage=0.15,        # 15% test (70% train)
    random_state=123,            # Different seed
    verify_images=True,          # Verify images exist
    min_annotations_per_image=2  # Only include images with 2+ annotations
)

print("🔍 Advanced Split (min 2 annotations per image):")
for split_name, split_stats in advanced_stats.items():
    if split_name != 'total':
        print(f"   {split_name.capitalize()}: {split_stats['images']} images, {split_stats['annotations']} annotations")


In [ ]:
# Cell 6: Verify Split Integrity
print("🔍 Verifying split integrity...")

verification = sv.verify_coco_split(
    train_annotations="dataset_splits/train_annotations.json",
    val_annotations="dataset_splits/val_annotations.json",
    test_annotations="dataset_splits/test_annotations.json",
    original_annotations="sample_dataset/annotations.json"
)

print("✅ Verification Results:")
checks = [
    ("Valid COCO Format", verification['valid_format']),
    ("No Duplicate Images", verification['no_duplicates']),
    ("Complete Split", verification['complete_split']),
    ("Categories Preserved", verification['categories_preserved'])
]

for check_name, result in checks:
    status = "✅ PASS" if result else "❌ FAIL"
    print(f"   {check_name}: {status}")

if verification['details']:
    print("\n📋 Detailed Statistics:")
    details = verification['details']
    print(f"   Original: {details.get('original_images', 'N/A')} images, {details.get('original_annotations', 'N/A')} annotations")
    for split in ['train', 'val', 'test']:
        img_key = f'{split}_images'
        ann_key = f'{split}_annotations'
        if img_key in details:
            print(f"   {split.capitalize()}: {details[img_key]} images, {details[ann_key]} annotations")


In [ ]:
# Cell 7: Examine Split Files
print("🔍 Examining generated split files...")

# Load and examine a split file
with open("dataset_splits/train_annotations.json") as f:
    train_data = json.load(f)

print("📄 Train Split Structure:")
print(f"   Info: {train_data['info']['description']}")
print(f"   Categories: {len(train_data['categories'])}")
print(f"   Images: {len(train_data['images'])}")
print(f"   Annotations: {len(train_data['annotations'])}")

# Show sample image from train split
if train_data['images']:
    sample_img = train_data['images'][0]
    print(f"\n🖼️ Sample Image: {sample_img['file_name']}")
    print(f"   Dimensions: {sample_img['width']}x{sample_img['height']}")
    
    # Count annotations for this image
    img_annotations = [ann for ann in train_data['annotations'] 
                      if ann['image_id'] == sample_img['id']]
    print(f"   Annotations: {len(img_annotations)}")



In [ ]:
# Cell 8: Category Distribution Analysis
print("📊 Analyzing category distribution across splits...")

def analyze_category_distribution(annotations_file, split_name):
    """Analyze category distribution in a split."""
    with open(annotations_file) as f:
        data = json.load(f)
    
    # Count annotations per category
    category_counts = {}
    for ann in data['annotations']:
        cat_id = ann['category_id']
        category_counts[cat_id] = category_counts.get(cat_id, 0) + 1
    
    # Get category names
    cat_id_to_name = {cat['id']: cat['name'] for cat in data['categories']}
    
    print(f"📈 {split_name} Category Distribution:")
    for cat_id, count in sorted(category_counts.items()):
        cat_name = cat_id_to_name.get(cat_id, f"Unknown_{cat_id}")
        print(f"   {cat_name}: {count} annotations")
    
    return category_counts

# Analyze all splits
splits_to_analyze = [
    ("dataset_splits/train_annotations.json", "Train"),
    ("dataset_splits/val_annotations.json", "Validation"),
    ("dataset_splits/test_annotations.json", "Test")
]

all_distributions = {}
for file_path, split_name in splits_to_analyze:
    if Path(file_path).exists():
        all_distributions[split_name] = analyze_category_distribution(file_path, split_name)
    print()

In [ ]:
# Cell 9: Best Practices and Tips
print("""
💡 Best Practices for COCO Dataset Splitting:

1. 🎯 **Consistent Random Seed**: Always use the same random_state for reproducible experiments
   
2. 🔍 **Verify Images**: Enable verify_images=True to catch missing or corrupted images early
   
3. 📊 **Check Distribution**: Ensure each split has sufficient examples of all categories
   
4. 💾 **Backup Original**: Keep your original annotations file safe before splitting
   
5. 🎛️ **Adjust Ratios**: Common splits are 70/15/15 or 80/10/10 (train/val/test)
   
6. 📝 **Minimum Annotations**: Use min_annotations_per_image to filter low-quality images
   
7. ✅ **Verify Splits**: Always run verify_coco_split() to ensure integrity
   
8. 📁 **Organized Structure**: Keep split files in separate directories for clarity
""")


In [ ]:
# Cell 10: Using Splits with Popular Frameworks
print("🔧 Example: Loading splits with popular frameworks...")

print("""
# PyTorch Example
from torch.utils.data import Dataset
from pycocotools.coco import COCO

class COCODataset(Dataset):
    def __init__(self, annotations_file, images_dir, transform=None):
        self.coco = COCO(annotations_file)
        self.images_dir = images_dir
        self.transform = transform
        self.image_ids = list(self.coco.imgs.keys())
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_info = self.coco.imgs[img_id]
        # ... load and process image ...

# Create datasets
train_dataset = COCODataset('dataset_splits/train_annotations.json', 'sample_dataset/images')
val_dataset = COCODataset('dataset_splits/val_annotations.json', 'sample_dataset/images')

# Supervision Integration
train_detections = sv.DetectionDataset.from_coco(
    images_directory_path='sample_dataset/images',
    annotations_path='dataset_splits/train_annotations.json'
)
""")

In [ ]:
# Cell 11: Clean Up (Optional)
cleanup_choice = input("🗑️ Clean up generated files? (y/N): ").lower().strip()

if cleanup_choice == 'y':
    import shutil
    
    # Remove generated directories
    dirs_to_remove = ['sample_dataset', 'dataset_splits', 'advanced_splits']
    
    for dir_path in dirs_to_remove:
        if Path(dir_path).exists():
            shutil.rmtree(dir_path)
            print(f"🗑️ Removed {dir_path}")
    
    print("✅ Cleanup complete!")
else:
    print("📁 Files preserved for further exploration")

print("""
🎉 Tutorial Complete!

You've learned how to:
✅ Split COCO datasets efficiently with Supervision
✅ Verify split integrity and distribution
✅ Handle edge cases and missing images
✅ Integrate with popular ML frameworks

📚 Next Steps:
- Try with your own COCO dataset
- Experiment with different split ratios
- Integrate with your training pipeline
- Explore other Supervision features

🔗 Resources:
- Supervision Documentation: https://supervision.roboflow.com/
- GitHub Repository: https://github.com/roboflow/supervision
- Community: https://github.com/roboflow/supervision/discussions
""")